In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

from PIL import Image, ImageOps, ImageFilter

# import imutils
# import itertools
# import shutil
# import random
# import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

### Processing Covid19 Patients ECG images

In [2]:
def gammaCorrection(src, gamma):
    invGamma = 1 / gamma
 
    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
 
    return cv2.LUT(src, table)

In [ ]:
main_path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1"

for foldername in os.listdir(main_path): # looping over the different class folder
    
    folder = main_path+"/"+foldername+"/"
    Copy_to_path = folder+"/processed/"

    for index, filename in enumerate(os.listdir(folder)): # retrieving the image files in particular folder
        
        if filename != 'processed':
            
            image = cv2.imread(folder+filename, cv2.IMREAD_GRAYSCALE) # grayscale

            enhance = cv2.equalizeHist(image) # equalize histogram used to enchance the lines

            thrsh = cv2.adaptiveThreshold(enhance,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,41,21) 

            contrs, _ = cv2.findContours(thrsh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) # using threshold to get the contours
            contrs = list(contrs)

            c = max(contrs, key = cv2.contourArea) # getting the largest contours that is the largest shape ( image boundary )

            print(c.shape)
            if c.shape[0] <= 4:            

                contrs.remove(max(contrs, key = cv2.contourArea)) # removing the frame

                c = max(contrs, key = cv2.contourArea) # again getting the largest frame that is the pulse frame

                x,y,w,h = cv2.boundingRect(c) # getting the co-ordinates


                rect = image[y:y+h, x:x+w] # cropping the frame

                cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),10) # drawing rectangle with the co-ordinates

                gamma_corrected = gammaCorrection(rect,3) # gamma correction

                (thresh, im_bw) = cv2.threshold(gamma_corrected, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # threshold is increased

                im_bw_resize = cv2.resize(im_bw,(720, 576)) # resizing the image to 256
                img = Image.fromarray(im_bw_resize)
                print(filename, " ", index)
                img.save(Copy_to_path+f'{foldername}_{index}.jpg', 'JPEG')
        else:
            pass

### Data Augmentation


In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, img_to_array

In [16]:
main_path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1"
augmented_path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1\\augmented"

data_gen_args = dict(
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     fill_mode='constant',
                     cval=255
                    )
image_datagen = ImageDataGenerator(**data_gen_args)

for foldername in os.listdir(main_path): # looping over the different class folder
    if foldername != 'augmented':
        
        folder = main_path+"\\"+foldername+"\\processed\\"

        Copy_to_path = augmented_path+'\\'+foldername+'\\'
        print(Copy_to_path)

    #     determining the files count in each category
    #     we want 7500 images in final dataset
    #     we have five categories
    #     so we need 1500 images in each category
        files_count = os.listdir(folder) 
        print(len(files_count))

        total_image_present = len(files_count)
        img_batch_size = round(2000/total_image_present)
        if img_batch_size > 8 :
            img_batch_size = 8

        print(f"batch size for {foldername} category is {img_batch_size}")
        file_count = 0
        for filename in os.listdir(folder): # retrieving the image files in particular folder

            img = Image.open(folder+filename)
            
            file_count += 1
            #PIL images into NumPy arrays
            array = img_to_array(img)
            reshaped_np_arr = array.reshape((1,)+array.shape)
            count = 0

    #             generating the augmentation of images
            for batch in image_datagen.flow(
                reshaped_np_arr,
                batch_size=1,
                save_to_dir=Copy_to_path,
                save_prefix = foldername+f'{file_count}',
                save_format='jpeg',
                seed=20
            ):

                count += 1
                if count == img_batch_size:
                    break
                    
                            
    else:
        pass

C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\abnormalHR\
538
batch size for abnormalHR category is 4
C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\covid19\
250
batch size for covid19 category is 8
C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\historyMI\
199
batch size for historyMI category is 8
C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\miPatients\
72
batch size for miPatients category is 8
C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\normal\
854
batch size for normal category is 2


### Test Train Split of all classes

In [1]:
!pip install split-folders

In [7]:
import splitfolders

In [8]:
augmented_folder = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1\\augmented"
output_folder = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1\\inputImages"

In [9]:
splitfolders.ratio(augmented_folder, output=output_folder,
    seed=28, ratio=(.75, .25), group_prefix=None, move=False)

Copying files: 8032 files [00:05, 1482.88 files/s]


# Reading the Test and Val images

In [30]:
import tensorflow as tf

In [31]:
#!pip install tensorflow-gpu

##### Reading Training Data using keras

In [4]:
train_folder = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1\\inputImages\\train"

training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_folder, 
    labels='inferred', 
    label_mode='int',
    color_mode='grayscale',
    batch_size=64,
    image_size=(720, 576),
    shuffle=False, seed=11,
    validation_split=None,
    subset=None,
    follow_links=False
)

Found 6024 files belonging to 5 classes.


##### Reading Test/val Data using keras

In [5]:
val_folder = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1\\inputImages\\val"

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_folder, 
    labels='inferred', 
    label_mode='int',
    color_mode='grayscale',
    batch_size=64,
    image_size=(720, 576),
    shuffle=False, seed=11,
    validation_split=None,
    subset=None,
    follow_links=False
)

Found 2008 files belonging to 5 classes.


In [6]:
type(training_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [7]:
count = 0
for image in training_ds.take(1):
    image
    count += 1
    if count == 1:
        break

# Model Creation and training

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [9]:
# img_rows, img_cols = 720, 576
# input_shape = (img_rows, img_cols, 1)

###  CNN with 3 Convolutional Layer

In [10]:
cnn3 = Sequential()
cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(720,576,1)))
cnn3.add(BatchNormalization())

cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn3.add(MaxPooling2D(pool_size=(2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(128, kernel_size=(2, 2), activation='relu'))
cnn3.add(Dropout(0.4))

cnn3.add(Flatten())

cnn3.add(Dense(128, activation='relu'))
cnn3.add(Dropout(0.3))
cnn3.add(Dense(5, activation='softmax'))

cnn3.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
cnn3.fit(training_ds,
         batch_size=64,
         epochs=11,
         verbose='auto',
         use_multiprocessing=True,
)

Epoch 1/11
 7/95 [=>............................] - ETA: 1:38:23 - loss: 0.6113 - accuracy: 0.8638

### CNN with 4 Convolutional Layer

In [ ]:
cnn4 = Sequential()
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn4.add(BatchNormalization())

cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.25))

cnn4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Flatten())

cnn4.add(Dense(512, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(128, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(5, activation='softmax'))

cnn4.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

# Testing and Junkies

### Do Not Run These unless testing something

In [13]:
augmented_path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data1\\augmented"
        

for foldername in os.listdir(augmented_path): # retrieving the image files in the particular folder
    
    folder = augmented_path+"\\"+foldername
    copy_path = augmented_path+'\\'+foldername+'\\cropped\\'
    print(copy_path)
    count = 0
    
    for filename in os.listdir(folder):
    
        if filename != 'cropped':
            count += 1
            image = cv2.imread(folder+"\\"+filename) #reading the images
            imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #converting the image to grayscale
            
            ret, thresh = cv2.threshold(imgray, 0, 1, 0) #finding image threshold
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            # finding all the contours using chain approx none, chain_approx_simple creating many no of contours
            
            contour_max = max(contours, key = cv2.contourArea) # finding the highest contour for cropping the image
            
            x,y,w,h = cv2.boundingRect(contour_max) # getting the co-ordinates
            image = image[y:y+h, x:x+w] # cropping the frame
            cv2.imwrite(copy_path + f'{foldername}_{count}.jpg', image)

        else:
            pass
        

C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\abnormalHR\cropped\
C:\Users\anant\jupyterBooks\LectureNotebooks\2022W\AML_2404_LAB\data1\augmented\covid19\cropped\


KeyboardInterrupt: 

In [1]:
import os
import numpy as np
import cv2

In [4]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array

## Testing Augmenting image code

In [10]:
path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data\\test"
os.chdir(path)
print(os.listdir(os.curdir))
filename = "abnormalHR_14"

img_batch_size = 8

data_gen_args = dict(
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     fill_mode='constant',
                     cval=255
                    )
image_datagen = ImageDataGenerator(**data_gen_args)


img = cv2.imread("abnormalHR_14.jpg")


reshaped_np_arr = img.reshape((1,)+img.shape)
count = 0

#             generating the augmentation of images
for batch in image_datagen.flow(
    reshaped_np_arr,
    batch_size=1,
    save_to_dir=os.curdir,
    save_prefix = filename,
    save_format='jpeg',
    seed=20
):

    count += 1
    if count == img_batch_size:
        break

['abnormalHR_0.jpg_0_840.jpeg', 'abnormalHR_0_cropped.jpeg.jpg', 'abnormalHR_124_cropped.jpeg.jpg', 'abnormalHR_14.jpg', 'abnormalHR_24.jpg_0_9181.jpeg', 'Screenshot 2022-03-28 222642.jpg']


In [6]:
path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data\\test"
os.chdir(path)
print(os.listdir(os.curdir))
img = cv2.imread('abnormalHR_0.jpg_0_840.jpeg')

imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 0, 1, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print(type(contours))
print(len(contours))

if len(contours) != 0:
        
    c = max(contours, key = cv2.contourArea)
    contour_image = cv2.drawContours(img, contours, -1, (255,0,0), 3)
    cv2.imshow("out", contour_image)
    cv2.waitKey(0)
    x,y,w,h = cv2.boundingRect(c)
        
    rect = contour_image[y:y+h, x:x+w]
    cv2.imshow("out", rect)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

['abnormalHR_0.jpg_0_840.jpeg']
<class 'tuple'>
2


In [3]:
path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data\\test"
os.chdir(path)
print(os.listdir(os.curdir))
filename = "abnormalHR_124_cropped.jpeg"
image = cv2.imread("abnormalHR_24.jpg_0_9181.jpeg") #reading the images

imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #converting the image to grayscale
ret, thresh = cv2.threshold(imgray, 0, 1, 0) #finding image threshold
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# finding all the contours using chain approx none, chain_approx_simple creating many no of contours
print(hierarchy)
print(len(contours))

contour_max = max(contours, key = cv2.contourArea) # finding the highest contour for cropping the image

x,y,w,h = cv2.boundingRect(contour_max) # getting the co-ordinates

image = image[y:y+h, x:x+w] # cropping the frame
cv2.imwrite(f'{filename}.jpg', image)

['abnormalHR_0.jpg_0_840.jpeg', 'abnormalHR_0_cropped.jpeg.jpg', 'abnormalHR_124_cropped.jpeg.jpg', 'abnormalHR_24.jpg_0_9181.jpeg', 'Screenshot 2022-03-28 222642.jpg']
[[[ -1  -1   1  -1]
  [  2  -1  -1   0]
  [  3   1  -1   0]
  ...
  [358 356  -1   0]
  [359 357  -1   0]
  [ -1 358  -1   0]]]
360


True

In [10]:
path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data\\test"
os.chdir(path)
print(os.listdir(os.curdir))
img = cv2.imread('abnormalHR_0.jpg_0_840.jpeg', cv2.IMREAD_GRAYSCALE)

# contours, _ = cv2.findContours(...) # Your call to find the contours using OpenCV 2.4.x
blur = cv2.GaussianBlur(img,(5,5),0)
ret, threshold = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contrs, _ = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) # using threshold to get the contours

contrs_list = list(contrs)
contrs_list.remove(max(contrs_list, key = cv2.contourArea)) # removing the frame
# again getting the largest frame that is the pulse frame

c = max(contrs_list, key = cv2.contourArea)


x,y,w,h = cv2.boundingRect(c) # getting the co-ordinates


rect = img[y:y+h, x:x+w] # cropping the frame

#                 cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),10) # drawing rectangle with the co-ordinates

#                 gamma_corrected = gammaCorrection(rect,3) # gamma correction

#                 (thresh, im_bw) = cv2.threshold(gamma_corrected, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # threshold is increased

#                 im_bw_resize = cv2.resize(im_bw,(720, 576)) # resizing the image to 256
# img = Image.fromarray(rect)


cv2.imshow('image',rect)
# cv2.drawContours(img, contrs, 3, (0,255,0), 3)
cv2.waitKey(0)
cv2.destroyAllWindows()

#thrsh = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,41,21) 

# canny_output = cv2.Canny(img, thrsh, thrsh * 2)
# contours, _ = cv2.findContours(thrsh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Your call to find the contours
# print(contours)

# # Draw contours
# drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
# for i in range(len(contours)):
#     color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
#     cv.drawContours(drawing, contours, i, color, 2, cv.LINE_8, hierarchy, 0)
# # Show in a window
# cv.imshow('Contours', drawing)



['abnormalHR_0.jpg_0_840.jpeg']
<class 'numpy.ndarray'>
(101, 1, 2)


In [ ]:
idx = 2 # The index of the contour that surrounds your object
mask = np.zeros_like(img) # Create mask where white is what we want, black otherwise
cv2.drawContours(mask, contours, idx, 255, -1) # Draw filled contour in mask
out = np.zeros_like(img) # Extract out the object and place into output image
out[mask == 255] = img[mask == 255]

# Show the output image
cv2.imshow('Output', out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
path = "C:\\Users\\anant\\jupyterBooks\\LectureNotebooks\\2022W\\AML_2404_LAB\\data\\test"
os.chdir(path)
print(os.listdir(os.curdir))
img = cv2.imread('abnormalHR_0.jpg_0_840.jpeg') # Read in your image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) 

kernel = np.ones((9,9), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# put mask into alpha channel of result
result = img.copy()
result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
result[:, :, 3] = mask

# save resulting masked image
#cv2.imwrite('retina_masked.png', result)
cv2.imshow('Output', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

['abnormalHR_0.jpg_0_840.jpeg']


In [ ]:
# PATH = "data/covid19/"


# for filename in os.listdir(PATH):
#     img = Image.open(os.path.join(PATH, filename)) # images are color images
#     img = img.resize((1920,1080), Image.BICUBIC)
#     img.save(Copy_to_path+filename) 

In [ ]:

# filename = 'data/covid19/Binder1_Page_001.jpg'

# size = (720, 576)
# count = 0
# for filename in g
# with Image.open(filename) as im:
#     gray_img = ImageOps.grayscale(im)
#     gray_img.save('data/covid19/processed/covid001'+'.jpeg')

In [ ]:
# file_path = "data/covid19/Binder1_Page_017"
# Copy_to_path="data/covid19/processed/"

# count = 0

# for filename in glob.glob(f'{file_path}.jpg'):
#     count += 1
#     with Image.open(filename) as img:
#         img = img.convert('L')
#         img = img.filter(ImageFilter.SHARPEN)
#         img = img.filter(ImageFilter.EDGE_ENHANCE)
        
#         #img = img.filter(ImageFilter.CONTOUR)    
#         #img = img.filter(ImageFilter.EMBOSS)
#         #img = img.filter(ImageFilter.EDGE_ENHANCE)
#         image = np.array(img) # converting PIL image to numpy array
        
#         thrsh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 41, 21) 

#         contrs, _ = cv2.findContours(thrsh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) # using threshold to get the contours
#         contrs = list(contrs)
        
#         c = max(contrs, key = cv2.contourArea)
        
#         contrs.remove(max(contrs, key = cv2.contourArea)) # removing the frame

#         c = max(contrs, key = cv2.contourArea) # again getting the largest frame that is the pulse frame

#         x,y,w,h = cv2.boundingRect(c) # getting the co-ordinates


#         rect = image[y:y+h, x:x+w] # cropping the frame

#         cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),10) # drawing rectangle with the co-ordinates

#         gamma_corrected = gammaCorrection(rect,3) # gamma correction

#         (thresh, im_bw) = cv2.threshold(gamma_corrected, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # threshold is increased
        
#         print(type(im_bw))
#         processed_img = Image.fromarray(im_bw) #Converting the array to image
#         processed_img.show()
#         processed_img.save(Copy_to_path+f'/covid19_{count}.jpg', 'JPEG') 
